# Using Genetic Algorithms to Form Optimal Groups

We have all been in that situation. You have to do a group project in a class, and the lecturer assigns the groups randomly. You find yourself in a group with people you don’t know and end up doing all the work by yourself. We want to change that and want you to implement an Genetic Algorithm that optimizes groups.


## Task 1:


You have to implement the initial population, crossover, mutation, and fitness function. We divided it into smaller subtasks for you and implemented some of the functions for you. If you want to do your own implementation, feel free to ignore our hints and functions.

Your initial population consists of 50 group distributions, where 100 students are assigned to 20 groups. Each student has a Student ID, a name, their spoken language, their 2 Majors (we assume everyone is doing their Masters), their ambition in the course, their preferred meeting place, their personality type, their gender, a friend that they want to be in a group with, and their preferred meeting day.

To get you even more involved with the task, we want every member of your group to add their own person into the data. To do so, you simply have to execute the Jupyter Notebook named "Dataset_Input.ipynb". We are using the Myers-Briggs personality, which may not be the most scientific, but it for sure is entertaining. If you don’t know your type, you can take the test here (approx. 10 minutes) https://www.16personalities.com/free-personality-test. 

Run this cell to load the packages:

In [11]:
import numpy as np
import random
import pandas as pd
import matplotlib as plt
from ast import literal_eval
import ipywidgets
import IPython
from Generator import DataGenerator
from typing import List

Execute this cell to create the dataset:

In [13]:
generator = DataGenerator(50)

# df, type_ = generator.create_basic()

df, type_ = generator.create_full()

df.to_csv("dataset_" + type_ + ".csv", encoding="utf-8-sig", index=False)

Hyperparameters:

In [45]:
students: pd.DataFrame = pd.read_csv("dataset_full.csv")
student_ids: List[int] = students.ID.tolist()

# hyperparameters
num_individuals: int = 100
groupsize: int = 5
# between 0 and 1
mutation_rate: float = 0.01

students.tail()

,ID,Name,Gender,Preferred language,Majors,Level of ambition,Preferred meeting place,Personality type,Best friend,Preferred day
45,46,Klaus Fried,Male,Any,"('NS', 'CL')",Low,In person,ESTJ,Maria Becker,Thursday
46,47,Susanne Keller,Female,Any,"('PHIL', 'CL')",Medium,In person,ESTP,Kristian Nacht,Monday
47,48,Manuela Lang,Female,Any,"('NS', 'NI')",Low,In person,ENFJ,Phillipp Bürger,Wednesday
48,49,Kristin Cole,Female,Any,"('CP', 'CL')",High,In person,INFJ,Marcel Cole,Tuesday
49,50,Tobias Herman,Male,Any,"('NI', 'NS')",Medium,In person,INFJ,Kristian Kortig,Friday


We create one random individual:

In [15]:
def create_random_individual(student_ids: List[int]) -> List[int]:
    #You don't need to do anything here.

    individual = student_ids.copy()
    random.shuffle(individual)

    return individual

print(create_random_individual(student_ids))

[13, 5, 47, 7, 31, 17, 14, 8, 20, 35, 42, 46, 37, 2, 16, 22, 29, 1, 18, 15, 12, 34, 49, 25, 26, 24, 3, 44, 33, 23, 48, 32, 43, 50, 4, 10, 9, 19, 27, 6, 41, 36, 21, 45, 30, 38, 39, 11, 28, 40]


Create the initial population of 50 (=num_individuals) individuals:

In [16]:
def create_initial_population(student_ids, groupsize: int) -> np.ndarray:
    #should return a numpy array of the whole population
    return np.array(sum([create_random_individual(student_ids) for _  in range(groupsize)], []), dtype=int)

We need a fitness function that computes how good the group distribution is. You have to take into consideration that we have many parameters that are not equally important and should therefore be differently weighted. We already coded some of the specific evaluation functions for the parameters for you. You have to implement the remaining evaluation functions. Think about what is desirable to have in a group and how you can calculate it. For example, you might have to change the data type of a parameter to make meaningful calculatons.

At the end, you have to weigh all evaluation functions in one fitness function. 

In [6]:
# def evaluate_language(group: pd.DataFrame) -> float:
#     preferred_language = group['Preferred language'].value_counts()
#     if len(preferred_language) > 0:
#         if preferred_language[0] == groupsize:
#             return 5
#     return 0

In [17]:
def evaluate_language(group: pd.DataFrame) -> float:
    # number of groupmembers per language
    counts = group['Preferred language'].value_counts()
    #if languages are conflicting, return -1
    if 'German' in counts.index and 'English' in counts.index:
        return -1
    #if there is no conflict, return +5
    return groupsize

In [18]:

def evaluate_majors(group: pd.DataFrame) -> float:
    # Remember that the majors are stored as a string like "('NS','AI')". 
    # You might need to preprocess those to actual tupels.
    majors = [literal_eval(major) for major in group['Majors']]
    majors = list(sum(majors, ()))
    same_majors = [majors.count(major) for major in majors]
    return sum(same_majors)


In [8]:
# def evaluate_ambition(group: pd.DataFrame) -> float:
#     #it might be useful to convert the strings to an integer scala
#     level_ambition = list(group['Level of ambition'])

#     if level_ambition.count("Very high") > int(groupsize/2) or level_ambition.count("High") > int(groupsize/2):
#         return 5
#     elif level_ambition.count("Medium") > int(groupsize/2):
#         return 3
#     return 1

In [19]:
def evaluate_ambition(group: pd.DataFrame) -> float:
    # Map ambitions to ineger from 0-4
    ambitions = ['Very low', 'Low', 'Medium', 'High', 'Very high']
    ambition_mappings = np.arange(5)
    for ambition, mapping in zip(ambitions, ambition_mappings):
        group = group.replace(ambition, mapping)

    # Return normalized with the maximally possible ambition level (16)
    return group['Level of ambition'].sum() / 16

In [20]:
def evaluate_meeting_place(group: pd.DataFrame) -> float:
    #You don't need to code anything in here.
    #This is an example evaluation function.
    
    # number of groupmembers for each preferred meeting place
    meeting_place = group['Preferred meeting place'].value_counts()
    if len(meeting_place)>0:
        # if all prefer the same meeting place return 5, else 0
        if meeting_place[0] == groupsize:
            return 5
    return 0

In [21]:
def evaluate_gender(group: pd.DataFrame) -> float:
    return 5

In [22]:
def evaluate_friends(group: pd.DataFrame) -> float:
    friends = list(group["Best friend"])
    names = list(group["Name"])
    num_of_friends = [1 for friend_a, friend_b in zip(friends, names) if friend_a == friend_b]
    return sum(num_of_friends)

In [23]:
def evaluate_personality(group: pd.DataFrame) -> float:
    #You dont need to change anything in here 

    #information about compatible personality types is taken from
    # Montequín, Vicente Rodríguez, et al. "Using Myers-Briggs type indicator (MBTI) as a tool for setting up student teams for information technology projects." Journal of Information Technology and Application in Education 1.1 (2012): 28-34.

    #count existing personality types in each group
    personalities = group['Personality type']
    types = personalities.value_counts()

    #fitness function starts with 0 and gets better
    # with every good group member
    fitness = 0

    #its good if there is a group leader like an ISTJ or an ESTJ, but only one
    try:
        if (types['ISTJ'] + types['ESTJ'] == 1):
            fitness+=5
        elif (types['ISTJ'] + types['ESTJ'] >= 2):
            fitness-=5
    except KeyError:
        pass

    #compare compatibility of group members
    for i, personality_a in enumerate(personalities.tolist()):
        for j, personality_b in enumerate(personalities.tolist()):
            # skip same group member and members already compared
            if i <= j:
                continue

            # increase fitness if
            if (personality_a[1] != personality_b[1]) ^ (personality_a[2] != personality_b[2]):
                if (personality_a[0] != personality_b[0]) or (personality_a[3] != personality_b[3]):
                    fitness+=1

    return fitness

In [24]:
def evaluate_meeting_day(group: pd.DataFrame) -> float:
    meeting_day = group['Preferred day'].value_counts()
    if len(meeting_day) > 0:
        # if all prefer the same meeting day return 5, else 0
        if meeting_day[0] == groupsize:
            return 5
    return 0

And now put everything together.
The function is almost done, but remember to add the weights.

In case you want to add hard constraints, feel free to do that in this function.



In [25]:
def evaluate_fitness(individual: np.ndarray, students: pd.DataFrame):
    # split individual into student groups of the groupsize
    # print("here ", individual)
    groups = np.array_split(individual, (len(individual)/groupsize))

    # iterate over groups and calculate scores for the different parameters
    scores = []
    for group_ids in groups:
        # get full data for students in this group from pd dataframe

        group = students.loc[students['ID'].isin(group_ids)]

        # get individual scores for parameters
        language_score = evaluate_language(group)
        major_score = evaluate_majors(group)
        ambition_score = evaluate_ambition(group)
        place_score = evaluate_meeting_place(group)
        gender_score = evaluate_gender(group)
        friend_score = evaluate_friends(group)
        personality_score = evaluate_personality(group)
        day_score = evaluate_meeting_day(group)

        # formula for adding and weighting different scores
        scores.append(language_score+major_score+ambition_score+place_score+gender_score+friend_score+personality_score+day_score)

    #Convert to series to calculate mean more easily
    return pd.Series(scores).mean()
    

Now, you have to code a crossover function, which takes 2 individuals based on their fitness function and produces a child from them. Use the tournament selection for parent selection.
For the crossover, we want you to use the uniform crossover function with random templates. 

In [26]:
def tournament_selection(population: np.ndarray, tournament_size: int) -> List[np.ndarray]:
    random.shuffle(population)
    splitted_arr = np.array_split(population, tournament_size)
    parent1, parent2 = 0, 0
    for i in range(0,len(splitted_arr)-2, 2):
        fitness_scores_candidate1 = evaluate_fitness(splitted_arr[i], students) 
        fitness_scores_candidate2 = evaluate_fitness(splitted_arr[i+1], students)

        if parent1 == 0 and parent2 == 0:
            parent1 = i
            parent2 = i+1
        
        if fitness_scores_candidate1 > parent1:
            parent1 = i
        elif fitness_scores_candidate2 > parent2:
            parent2 = i+1
    return splitted_arr[parent1], splitted_arr[parent2]

Use a boolean template in the length of the individual, this can be hardcoded or generated randomly every time to add more variance. On the places where the template is true use the genes from parent1, then take all the genes from parent2 that are not used and add them to the empty places in the child in the same order as they appear on parent2

In [27]:
def uniform_order_crossover(parent1: np.ndarray, parent2: np.ndarray, template: np.ndarray) -> np.ndarray:
    genes_of_parent1 = np.argwhere(template==True)
    genes_of_child = np.empty(parent1.shape)

    for idx_parent1 in genes_of_parent1:
        genes_of_child[idx_parent1] = parent1[idx_parent1]

    empty_places = np.argwhere(genes_of_child==0)
    counter = 0

    for empty_place in empty_places:
        genes_of_child[empty_place] = parent2[counter]
        counter+=1
    return genes_of_child

The last thing you need is the mutation function. It should take the individual produced by the crossover function and mutate it with a chance of for example 5%. A mutation switches the assigned groups of 2 people. 

In [28]:
def remove_duplicates(individual):
    if set(individual) == len(individual):
        return individual
    individual = list(individual)
    return [id if individual.count(id) == 1 else id - 2*idx for idx, id in enumerate(individual)]

In [51]:
def mutation(individual: np.ndarray, mutation_rate: float) -> np.ndarray:
    num_of_mutated_genes = int(individual.shape[0]/(mutation_rate*100))
    mutated_indices = random.sample(range(individual.shape[0]), num_of_mutated_genes)
    mutated_individual = individual.copy()
    for idx in mutated_indices:
        if mutated_individual[idx] > 0:
            mutated_individual[idx] -= random.randint(num_individuals)

    return remove_duplicates(mutated_individual)

You can now execute the code below and see if everything is working.

In [54]:
#episodes is the number of episodes after the algorithm stops
#num_replace is the number of unfit individuals that will be replaced
def genetic_algorithm(episodes: int, num_replace: int) -> np.ndarray:
    population = create_initial_population(list(students.ID), groupsize)
    
    new_population = []
    for episode in range(episodes):
        # since we are choosing two parents each tournament, we iterate over the half size of population
        parent1, parent2 = tournament_selection(population, 50)
        splitted_population = np.array_split(population, 50)

        population_fitness = [sum(individual)/len(individual) for individual in splitted_population]
        population_fitness_average = round(sum(population_fitness)/len(population_fitness),4)
        
        template = np.random.choice(a=[False, True], size=parent1.shape)
        child = uniform_order_crossover(parent1, parent2, template)
        child_mutated = list(np.array(mutation(child, mutation_rate), dtype=int))
        
        if episode % 50 == 0:
            print(f"Episode {episode}:")
            print(f"At the start, old population of size {len(splitted_population)} with average of the fitness score {population_fitness_average} and maxinmum fitness score {max(population_fitness)}")
            print("Fitnesss score of parent 1: ", evaluate_fitness(parent1, students))
            print("Fitnesss score of parent 2: ", evaluate_fitness(parent2, students))
            print("Fitnesss score of child: ", evaluate_fitness(child, students))
            print("Fitnesss score of mutated child: ", evaluate_fitness(child_mutated, students), '\n')

        # new_population.append(list(parent1))
        # new_population.append(list(parent2))
        new_population.append(child_mutated)

    print("Now we have a population of size ", len(new_population))

    population_fitness = [evaluate_fitness(individual, students) for individual in new_population]
    population_fitness_idx = np.argsort(population_fitness)

    to_replace = num_replace if len(population_fitness_idx) > num_replace else len(population_fitness_idx)
    tmp = []
    
    for i in range(to_replace):
        x = new_population[population_fitness_idx[i]]
        tmp.append(x)

    for x in tmp:
        new_population.remove(x)
    
    population_fitness = [sum(individual)/len(individual) for individual in new_population]
    population_fitness_average = round(sum(population_fitness)/len(population_fitness),4)
    print(f"After removing the weak elements, we have a population of size {len(new_population)} with average of the fitness score {population_fitness_average} and maxinmum fitness score {max(population_fitness)}")

In [55]:
population = genetic_algorithm(250,0)

Episode 0:
At the start, old population of size 50 with average of the fitness score 25.5 and maxinmum fitness score 38.0
Fitnesss score of parent 1:  42.6875
Fitnesss score of parent 2:  45.5
Fitnesss score of child:  30.375
Fitnesss score of mutated child:  41.5 

Episode 50:
At the start, old population of size 50 with average of the fitness score 25.5 and maxinmum fitness score 38.8
Fitnesss score of parent 1:  44.25
Fitnesss score of parent 2:  40.5
Fitnesss score of child:  38.375
Fitnesss score of mutated child:  24.25 

Episode 100:
At the start, old population of size 50 with average of the fitness score 25.5 and maxinmum fitness score 34.8
Fitnesss score of parent 1:  39.8125
Fitnesss score of parent 2:  41.8125
Fitnesss score of child:  39.875
Fitnesss score of mutated child:  30.6875 

Episode 150:
At the start, old population of size 50 with average of the fitness score 25.5 and maxinmum fitness score 35.8
Fitnesss score of parent 1:  36.625
Fitnesss score of parent 2:  44

### Let's find your group or the one of a friend

In [31]:
def get_groups_from_individual(individual: np.ndarray) -> List[np.ndarray]:
    """ Returns a Python List of all Groups in an individual(as numpy ndarrays) """
    groups = pd.DataFrame()
    for i in individual:
        groups = pd.concat([groups, students.loc[students['ID'] == i]])

    groups = groups.reset_index(drop=True)
    nested_groups_list = []
    for n,j in enumerate(range(0, len(groups), groupsize)):
        nested_groups_list.append(pd.DataFrame(groups[j:j+groupsize]))
    return nested_groups_list

def get_groups_by_person_fullname(fullname: str, individual: np.ndarray) -> List[np.ndarray]:
    """ Returns all Groups that got a Person with $fullname$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)

    groups_with_person = []
    for group in nested_groups_list:
        if fullname in group["Name"].values:
            groups_with_person.append(group)
    return groups_with_person

def get_group_by_person_ID(ID: int, individual: np.ndarray) -> np.ndarray:
    """ Returns Group that has Person with given $ID$ in given $individual$"""
    nested_groups_list = get_groups_from_individual(individual)
    groups_with_person = []
    for group in nested_groups_list:
        if ID in group["ID"].values:
            return group
    return None


#### Find your group by your Fullname

In [ ]:
######### ATTENTION ##########
# Define your individual here!!! At the moment it will take a random individual! Pick for example the fittest individual.
individual = create_random_individual(student_ids)

# User Input for the Full Name - Please be accurate!
fullname = str(input("Reem X"))
# Get the group by your Full Name - If there are multiple Persons with your name you will get all of them
my_group = get_groups_by_person_fullname(fullname, individual)
# Plotting the first group found
if my_group:
    print(my_group)
else:
    print("Didn't find the requested Person with given Fullname")

#### Find your group by your ID

In [146]:
# Find your group by your ID
my_group = get_group_by_person_ID(101, individual)
# Plotting the group, if one found with given ID
if my_group is not None:
    print(my_group)
else:
    print("Didn't find the requested Person with given ID")

     ID             Name  Gender Preferred language          Majors  \
75   48    Torsten Beyer  Female                Any  ('CP', 'PHIL')   
76   72     Karolin Bach  Female                Any  ('CL', 'PHIL')   
77   38   Heike Schwartz  Female                Any    ('NI', 'NS')   
78  101      Donald Dump  Divers                Any    ('AI', 'NI')   
79   61  Leon Zimmermann    Male            English  ('AI', 'PHIL')   

   Level of ambition Preferred meeting place Personality type  \
75              High               In person             INTP   
76               Low               In person             ENFJ   
77               Low               In person             INFP   
78            Medium                  Online             ENTP   
79               Low               In person             ENTJ   

             Best friend Preferred day  
75  Florian Baumgaertner      Thursday  
76     Michael Eberhardt     Wednesday  
77        Karin Glockner      Thursday  
78           Danie

## Task 2

Play around with the different values like initial population size, mutation rate, fitness function, and number of students and observe when it works the best. Write your insights down here.

### Population size:
- "large": 500
- "small": 50

### Mutation rate:
- 0.03
- 0.3

### Number of students:

